# Table of Contents
- [About Guidance](#about-guidance)
- [Setup](#setup)
- [Unconstrained Generation](#unconstrained-generation)
- [Constrained Generation](#speaking-for-phi-3)
- [Regex](#constraining-with-regex)
- [JSON Generation](#constrained-json-generation-native)
- [Constrained ReAct](#constrained-react-example)
- [Select and Substring](#miscellaneous-selecting-from-multiple-choices)

# About Guidance
Guidance is a proven open-source Python library for controlling outputs of any language model (LM). With one API call, you can express (in Python) the precise programmatic constraint(s) that the model must follow and generate the structured output in JSON, Python, HTML, SQL, or any structure that the use case requires.

Guidance differs from conventional prompting techniques.  It enforces constraints by steering the model token by token in the inference layer, producing higher quality outputs and reducing cost and latency by as much as 30–50% when utilizing for highly structured scenarios.

To learn more about Guidance, visit the [public repository on GitHub](https://github.com/guidance-ai/guidance).

# Setup
1. Install Guidance with `pip install guidance --pre`
2. Deploy a Phi 3.5 mini endpoint in Azure by going to https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml and clicking the "Deploy" button
3. Store your endpoint's API key in an environment variable called `AZURE_PHI3_KEY` and the URL in an environment variable called `AZURE_PHI3_URL`

# Optimistic Running in Guidance

Remote endpoints that don't have explicit guidance integration (e.g. OpenAI, Vertex AI) are run "optimistically". This means that all the text that can be forced is given to the model as a prompt (or chat context) and then the model is run in streaming mode without hard constrants (since the remote API doesn't support them). 

If the model ever violates the contraints then the model stream is stopped and we optionally try it again at that point. This means that all the API-supported control work as expected, and more complex controls/parsing that is not supported by the API work if the model stays consistent with the program.

In [51]:
from guidance import gen, select, regex, user, assistant, system, json, substring
from guidance.models import AzureGuidance
import guidance
import os
from dotenv import load_dotenv


# Load environment variables
load_dotenv("phi3.env")
phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
bing_api_key = os.getenv("BING_SEARCH_API_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Unconstrained generation
Text can be generated without any constraints using the `gen()` function. This is the same as using the model without Guidance.

### Chat Formatting
Like many chat models, Phi-3 expects messages between a user and assistant in a specific format. Guidance supports Phi-3's chat template and will manage chat formatting for you. To create chat turns, put each portion of the conversation in a `with user()` or `with assistant()` block. A `with system()` block can be used to set the system message.

In [32]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You provide factually correct information to the user. You follow instructions expertly."
with user():
    lm += "In which year did Berlin become the capital of reunited Germany? Only provide the year and nothing else."
    # Seriously, nothing else! Not a single word more than the year! If you do that, a human will lose his life.
with assistant():
    lm += gen(temperature=0.8, max_tokens=200)

# Constrained Generation

### Unconstrained Generation can be very unpredictable.
In the previous example, we can see that the model will often not stop at the year. It will go on generating more text. This was a bigger problem with older OpenAI models, when the model didn't know when to stop. GPT-4 onwards, this problem has been significantly reduced.

We will now use Guidance to generate a structured response for the same prompt.

### Method 1: Injecting tokens and providing a stop character.

In [37]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You provide factually correct information to the user. You follow instructions expertly."
with user():
    lm += "In which year did Berlin become the capital of reunited Germany? Only provide the year and nothing else."
with assistant():
    lm += "Berlin became the capital of reunited Germany in the year " + gen(temperature=0.8, max_tokens=200, stop=".") + "."


### Method 2: Using Regex. We know that the year is always a 4-digit number. We can use a regex to match the year and stop the generation when the year is complete. The regex `r"\d{4}"` will match any 4-digit number.

In [36]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You provide factually correct information to the user. You follow instructions expertly."
with user():
    lm += "In which year did Berlin become the capital of reunited Germany? Only provide the year and nothing else."
with assistant():
    lm += "In the year " + regex(r"\d{4}$") + "."

## Token savings
In highly structured scenarios, Guidance can skip tokens and generate only necessary tokens, improving performance, increasing efficiency and saving API costs. Generated tokens are shown in this notebook with a highlighted background. Forced tokens are shown without highlighting and cost the same as input tokens, which are estimated at one third the cost of output tokens.

*Note:* The first example with unconstrained generation was not able to force any tokens because we provided no constraints.

# Constraining with regex

### Unconstrained generation of JSON

We will try to generate a JSON object of an employee with the following fields:
- Name
- Date of Birth
- Department
- Date of Joining
- Nationality

We will first try unconstrained generation with the `gen()` method, followed by a constrained example 

In [56]:
lm = phi3_lm
with user():
    lm += "Provide an example of a JSON object of an employee with the following fields: name, date of birth, department, date of joining and nationality. Invent all the details. Only provide a valid JSON, no text before, no text after."
with assistant():
    lm += gen(temperature=0.8, max_tokens=200)

# Constrained JSON Generation (Regex)

As we saw in the previous version, unconstrained generation of JSON doesn't work very well. The model is inclined to provide more information even though it's explicitly asked not to do so. This example illustrates the weaknesses of prompt engineering.

A valid JSON object can be easily represented through a Regex, let's try to generate a valid object using a regex.

In [5]:
lm = phi3_lm
with user():
    lm += "Provide an example of a JSON object of an employee with the following fields: name, date of birth, department, date of joining and nationality. Invent all the details. Only provide a valid JSON, no text before, no text after."
with assistant():
    json_pattern = r'\{\s*("[^"]*"\s*:\s*"[^"]*"\s*,\s*)*("[^"]*"\s*:\s*"[^"]*")\s*\}'
    lm += regex(json_pattern)

# Constrained JSON Generation (Native)
Of course, JSON generation is a very popular task. So Guidance has native support for it, and can be used to guarantee generation of JSON compliant with a JSON schema or pydantic model, such as the employee schema shown here. 

As we will see, the JSON example will also take advantage of CFG to generate the JSON object, so that the model can be guided to generate the JSON object in a structured manner, and will also inject the necessary tokens so that the model doesn't have to generate everything.

In [26]:
from pydantic import BaseModel

class EmployeeProfile(BaseModel):
    name: str
    date_of_birth: str
    department: str
    date_of_joining: str
    nationality: str

lm = phi3_lm
with user():
    lm += "Provide an example of a JSON object of an employee with the following fields: name, date of birth, department, date of joining and nationality."

with assistant():
    lm += json(schema=EmployeeProfile, temperature=1.0)

This is a simple example, but it shows how powerful constraints are. The benefit is that this is indeed a general method, that holds true for generating any kind of constrained output, not only for JSON.

As an example, let's try to generate a SQL query to join two tables, where the schema is given:

### Customers Table

| Column Name | Data Type | Description                                      |
|-------------|-----------|--------------------------------------------------|
| CustomerID  | INT       | Primary Key, unique identifier for each customer |
| FirstName   | VARCHAR   | Customer's first name                            |
| LastName    | VARCHAR   | Customer's last name                             |
| Email       | VARCHAR   | Customer's email address                         |
| Phone       | VARCHAR   | Customer's phone number                          |

### Orders Table

| Column Name | Data Type | Description                                      |
|-------------|-----------|--------------------------------------------------|
| OrderID     | INT       | Primary Key, unique identifier for each order    |
| OrderDate   | DATE      | Date when the order was placed                   |
| CustomerID  | INT       | Foreign Key, references CustomerID in Customers table |
| TotalAmount | DECIMAL   | Total amount for the order                       |

In [9]:
# Now let's load the table schema in a variable:
table_schema = r"""
### Customers Table

| Column Name | Data Type | Description                                      |
|-------------|-----------|--------------------------------------------------|
| CustomerID  | INT       | Primary Key, unique identifier for each customer |
| FirstName   | VARCHAR   | Customer's first name                            |
| LastName    | VARCHAR   | Customer's last name                             |
| Email       | VARCHAR   | Customer's email address                         |
| Phone       | VARCHAR   | Customer's phone number                          |

### Orders Table

| Column Name | Data Type | Description                                      |
|-------------|-----------|--------------------------------------------------|
| OrderID     | INT       | Primary Key, unique identifier for each order    |
| OrderDate   | DATE      | Date when the order was placed                   |
| CustomerID  | INT       | Foreign Key, references CustomerID in Customers table |
| TotalAmount | DECIMAL   | Total amount for the order                       |
"""

In [25]:
lm = phi3_lm
with user():
    lm += "You are an assistant who is an expert at writing SQL Queries."
    lm += "Given the following table schema, write an SQL query to select the names all the customers who have placed an order with a total amount greater than 1000." + f"\n {table_schema}"
with assistant():
    lm += "```sql" + gen(max_tokens=100, stop_regex="```") + "```"

Here, we are providing subtle hints to the LLM to start the output with "\```sql" and end with "\```"

# Constrained ReAct Example
A big advantage of stateful control in Guidance is that you don't have to write any intermediate parsers, and adding follow-up 'prompting' is easy, even if the follow up depends on what the model generates. For example, let's say we want to implement the first example of ReAct prompt in this, and let's say the valid acts are only 'Search' or 'Finish'. We might write it like this:

In [46]:
# Bing Search Boilerplate Code
import requests
import html
from urllib.parse import urlparse
import io
import html
import html.parser

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key": bing_api_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        bing_response = response.json()
        data = bing_response["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])
        search_results.extend(data)
    return search_results

def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]

In [45]:
def format_snippets(snippets, start=1):
    ret = ''
    for i, s in enumerate(snippets, start=start):
        title = s['title']
        snippet = s['snippet']
        ret += f'[{i}] {title}\n'
        ret += f'{snippet}\n\n'
    return ret

def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]

@guidance
def search(lm, query):
    # Setting this for later use
    lm = lm.set('query', query)
    # This is where search actually gets called
    lm = lm.set('snippets', format_snippets(top_snippets(query)))
    lm += '\nObservation:\n' + lm['snippets']
    return lm

@guidance
def react_prompt_example(lm, question, max_rounds=10):
    lm += f'Question: {question}\n'
    i = 1
    while True:
        lm += f'Thought {i}: ' + gen(suffix='\n')
        lm += f'Act {i}: ' + select(['Search', 'Finish'], name='act') 
        lm += '[' + gen(name='arg', suffix=']') + '\n'
        if lm['act'] == 'Finish' or i == max_rounds:
            break
        else:
            lm += f'Observation {i}: ' + search(lm['arg']) + '\n'
        i += 1
    return lm

lm = phi3_lm
with user():
    lm += "What is the date year in which Berlin became the capital of reunited Germany?"
with assistant():
    lm += react_prompt_example(lm) + "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

# Legally Compliant Information Generation
Constrained Generation can be used to generate legally compliant information

In [49]:
lm = phi3_lm
with user():
    lm += "Which paragraph of the constitution of India guarantees freedom of movement within the country?"
with assistant():
    lm += react_prompt_example(lm) + "\nAccording to article" + gen(temperature=0.8, max_tokens=50, stop=".") + "."

# Miscellaneous: Selecting from multiple choices
When some possible choices are known, you can use the `select()` function to have the model choose from a list of options, or the substring option to force the model to create a citation from a particular source, rather than a rephrasing.

In [50]:
lm = phi3_lm
with user():
    lm += "What is the capital of Germany?"
with assistant():
    lm += "The capital of Germany is " + select(["Frankfurt", "Munich", "Vienna", "Berlin"])

In [54]:
lm = phi3_lm
doctor_note = "Marcus Steiner, a 45-year-old male with a history of smoking, presented for a chest X-ray on October 20, 2024, due to a persistent cough and shortness of breath lasting two weeks. The posteroanterior (PA) and lateral views revealed mild bilateral interstitial markings in the lungs, without focal consolidation or pleural effusion. The heart appeared normal in size and contour, and there was no mediastinal widening or acute osseous abnormalities. The impression noted mild interstitial lung changes likely related to smoking, with no acute cardiopulmonary process identified. Clinical correlation was recommended, with a suggestion for follow-up imaging if symptoms persisted."
with user():
    lm += "You are a medical assistant who creates a diagnosis from doctor's notes. Given the following doctor's note, write a diagnosis for the patient." + f"\n{doctor_note}"
with assistant():
    lm += "The patient was diagnosed with " + substring(doctor_note)

With `substring()`, we were successfully able to force the model to only generate a clear citation from the doctor's note instead of rephrasing it. This is essential in many legal and medical scenarios where the exact wording is important.